In [2]:
import pickle
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt


In [3]:
import wandb

wandb.login()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)


True

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")
    

Running on the GPU


In [5]:
REBUILD_DATA = True


class GRV:
    # class to store training data

    # reading data from pickle
    file = open(r"DATA/total_list_w_price_w_weights.npy", "rb")
    data = pickle.load(file)
    # seperate out classes from inputs
    raceIDs, inputs, classes, prices, win_price, margins = zip(*data)
    # removing nan from inputs and convert to float
    inputs_df = pd.DataFrame(inputs)
    inputs_df.fillna(value=-1, inplace=True)
    inputs = inputs_df.values.tolist()
    inputs = [[float(i) for i in j] for j in inputs]

    # data
    training_data = []

    def make_training_data(self):
        excluded = 0
        for i in range(len(self.inputs)):
            if len(self.classes[i]) == 8:
                self.training_data.append(
                    [
                        np.array(self.inputs[i]),
                        np.array(self.classes[i]),
                        np.array(self.prices[i]),
                        np.array(self.margins[i]),
                    ]
                )
            else:
                adjustedList = self.classes[i] + ([8] * (8 - len(self.classes[i])))
                adjustedListP = self.prices[i] + ([0] * (8 - len(self.prices[i])))
                adjustedListM = self.margins[i] + ([100] * (8 - len(self.margins[i])))
                self.training_data.append(
                    [
                        np.array(self.inputs[i]),
                        np.array(adjustedList),
                        np.array(adjustedListP),
                        np.array(adjustedListM),
                    ]
                )
                if len(adjustedList) != 8:
                    print(adjustedList)
        np.save("training_data.npy", self.training_data)
        print("excluded = ", excluded)


if REBUILD_DATA:
    grv = GRV()
    grv.make_training_data()


c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


excluded =  0


In [6]:
softmin = nn.Softmin()

pp = pprint.PrettyPrinter(indent=4)


pp.pprint(Y_m[0])

softmax(Y_m[1])


NameError: name 'Y_m' is not defined

In [7]:
softmin = nn.Softmin()

# dataset setup
training_data = grv.training_data

X = torch.Tensor([i[0] for i in training_data])
Y = torch.Tensor([i[1] for i in training_data])
P = torch.Tensor([i[2] for i in training_data])
Y_m = softmin(torch.Tensor([i[3] for i in training_data]))
# Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)

Y_w = torch.tensor([i for i in Y_w])
X = X.to(device)
Y_w = Y_w.to(device)


Y_m = Y_m.to(device)
P = P.to(device)
my_dataset = TensorDataset(X, Y_m, P)
my_dataloader = DataLoader(my_dataset)


C:\Users\Nick\AppData\Local\Temp\ipykernel_24580\2075100870.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  X = torch.Tensor([i[0] for i in training_data])
C:\Users\Nick\AppData\Local\Temp\ipykernel_24580\2075100870.py:9: UserWarning: Implicit dimension choice for softmin has been deprecated. Change the call to include dim=X as an argument.
  Y_m = softmin(torch.Tensor([i[3] for i in training_data]))


In [8]:
Y_m[0]


tensor([1.1902e-08, 5.4522e-01, 1.6185e-07, 4.3319e-01, 8.1155e-04, 3.9502e-04,
        2.0311e-02, 7.0031e-05], device='cuda:0')

In [ ]:
X[0].shape


torch.Size([120])

In [ ]:
np.pad(grv.prices, (0, 8), mode="constant")


c:\Users\Nick\.conda\envs\pytorch\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


array([list([49.2, 11.0, 53.4, 6.5, 4.8, 2.8, 3.3, 41.6]),
       list([27.3, 1.9, 18.9, 5.2, 5.4, 29.7, 10.2, 13.4]),
       list([4.5, 33.0, 34.6, 1.8, 6.1, 14.0, 15.4, 22.3]), ..., 0, 0, 0],
      dtype=object)

In [ ]:
config = dict(
    epochs=50,
    classes=8,
    kernels=[16, 32],
    batch_size=128,
    learning_rate=0.0001,
    validation_split=0.1,
    dataset="MNIST",
    architecture="CNN",
    dataset_size=len(training_data),
)

config["dataset_size"]


67100

In [9]:
# have to pass in dataset to get_data (created above)
def make_loader(dataset, config, train=True):
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(config["validation_split"] * dataset_size))
    random_seed = 42
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    if train:
        dataset_sampler = SubsetRandomSampler(train_indices)
        loader = torch.utils.data.DataLoader(
            dataset=dataset,
            batch_size=config["batch_size"],
            shuffle=False,
            pin_memory=False,
            num_workers=0,
            sampler=dataset_sampler,
        )
    else:
        dataset_sampler = SubsetRandomSampler(val_indices)
        loader = torch.utils.data.DataLoader(
            dataset=dataset,
            shuffle=False,
            pin_memory=False,
            num_workers=0,
            sampler=dataset_sampler,
        )

    return loader


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(81, 256)
        self.fc2 = nn.Linear(256, 32)
        self.fc3 = nn.Linear(32, 8)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return nn.Softmax(x, dim=1)
    

In [10]:
def build_network(f1_layer_size, f2_layer_size, dropout):
    # network = nn.Sequential(  # fully-connected, single hidden layer
    #     nn.Linear(96, f1_layer_size), nn.ReLU(),
    #     nn.Linear(f1_layer_size, 8), nn.Softmax(dim=1))

    network = nn.Sequential(  # fully-connected, dual hidden layer
        nn.Linear(120, f1_layer_size),
        nn.ReLU(),
        nn.Linear(f1_layer_size, f2_layer_size),
        nn.ReLU(),
        nn.Linear(f2_layer_size, 8),
    )

    return network


In [11]:
def make(config, dataset):
    # Make the data

    train_loader = make_loader(dataset, config, train=True)
    test_loader = make_loader(dataset, config, train=False)
    # Make the model
    # model = Net().to(device)
    model = build_network(
        config["f1_layer_size"], config["f2_layer_size"], config["dropout"]
    )

    # Make the loss and optimizer
    # criterion = nn.HuberLoss(reduction='sum', delta=1.0)
    # criterion = nn.MSELoss()
    # criterion = nn.CrossEntropyLoss()
    criterion = nn.NLLLoss()

    optimizer = config["optimizer"]

    if optimizer == "sgd":
        optimizer = optim.SGD(
            model.parameters(), lr=config["learning_rate"], momentum=0.9
        )
    elif optimizer == "adam":
        optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
    # optimizer = torch.optim.Adam(
    #    model.parameters(), lr=config["learning_rate"])

    return model, train_loader, test_loader, criterion, optimizer


In [12]:
def test(model, test_loader, batch_ct):
    model.eval()
    classL, predL, maxL, correctL, priceP, priceR = [], [], [], [], [], []
    # Run the model on some test examples
    with torch.no_grad():
        correct, total, max_sum, max_w_sum, profit = 0, 0, 0, 0, 0
        value_pick_correct, value_pick_profit = 0, 0
        num_bets = 0
        for images, labels, prices in test_loader:

            outputs = model(images)
            prices = prices[
                0,
            ].tolist()
            # print(f"{images=},\n,{labels=},\n{prices=}")

            max, predicted = torch.max(outputs.data, 1)
            _, real = torch.max(labels.data, 1)

            prediction = predicted.item()
            real_item = real.item()

            predL.append(prediction)
            maxL.append(max.item())

            total += labels.size(0)
            correct += prediction == real_item

            correctL.append(int(prediction == real_item))
            classL.append(real_item)

            priceR.append(prices[real_item])
            priceP.append(prices[prediction])
            # print(outputs.data.flatten().tolist())

            predicted_odds = [
                1 / ((x + 1**-7) / 1) for x in outputs.data.flatten().tolist()
            ]

            if prices[real_item] > (predicted_odds[real_item] * 1.2):
                value_pick_correct += 1
                value_pick_profit += prices[real_item]

            bets = [x > (y * 1.2) for x, y in zip(prices, predicted_odds)]
            num_bets += sum(bets)

            value_pick_profit += -sum(bets)

            if prediction == real_item:
                max_sum += max
                profit += prices[real_item]
            else:
                max_w_sum += max

            profit += -1

            # print(f"{correct=}")

        # print(f"Accuracy of the model on the {total} " +
        #       f"test images: {100 * correct / total}%" +
        #       f"profit: {profit}"+
        #       f"profit: {value_pick_profit}")

        wandb.log(
            {
                "test_accuracy": correct / total,
                "correct_conf": max_sum / correct,
                "incorrect_conf": (max_w_sum) / (total - correct),
                "profit": profit,
                "value_pick_roi": value_pick_profit / num_bets,
                "num_bets_per": num_bets / total,
            }
        )

        logdf = pd.DataFrame(
            data={
                "class": classL,
                "pred": predL,
                "max": maxL,
                "correct": correctL,
                "priceR": priceR,
                "priceP": priceP,
                "bets": sum(bets),
            }
        )
        table = wandb.Table(dataframe=logdf)
        wandb.log({"table_key": table})
        classCounts = logdf["class"].value_counts()
        predCounts = logdf["class"].value_counts()
        # boxplot = logdf.boxplot(column=['priceR'],by='correct')

        print(classCounts, predCounts)
        # boxplot
        # plt.savefig("boxplot.png")
        # wandb.log({"boxplot":boxplot})

    # Save the model in the exchangeable ONNX format
    # torch.onnx.export(model, images, "model.onnx")
    # wandb.save("model.onnx")


In [ ]:
def train(model, loader, test_loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
        for _, (images, labels, _) in enumerate(loader):

            loss = train_batch(
                images, labels, model, optimizer, criterion, btch_count=batch_ct
            )
            example_ct += len(images)
            batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 250) == 0:
                train_log(loss, example_ct, epoch)

        if epoch % 20 == 0:
            test(model, test_loader, batch_ct)


def train_batch(images, labels, model, optimizer, criterion, btch_count=0):
    images, labels = images, labels

    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels.float())

    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss


In [ ]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    # print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")


In [ ]:
def model_pipeline(config=None):
    dataset = my_dataset
    # tell wandb to get started
    with wandb.init(project="debug", config=config):
        # access all HPs through wandb.config, so logging matches execution!

        config = wandb.config
        pprint.pprint(config)
        pprint.pprint(config.epochs)
        print(config)

        # make the model, data, and optimization problem
        model, train_loader, test_loader, criterion, optimizer = make(config, dataset)
        model = model.to(device)
        print(model)

        # and use them to train the model
        train(model, train_loader, test_loader, criterion, optimizer, config)

        # and test its final performance
        test(model, test_loader)

    return model


In [ ]:
sweep_config


NameError: name 'sweep_config' is not defined

In [ ]:
CUDA_LAUNCH_BLOCKING = 1


normal_config = dict(
    epochs=2000,
    classes=8,
    f1_layer_size=32,
    f2_layer_size=32,
    batch_size=64,
    learning_rate=0.0005,
    optimizer="adam",
    loss="NLL",
    dropout=0.2,
    validation_split=0.2,
    dataset="MNIST",
    architecture="CNN",
    dataset_size=len(training_data),
)


model = model_pipeline(config=normal_config)


  0%|          | 0/2000 [00:00<?, ?it/s]

{'epochs': 2000, 'classes': 8, 'f1_layer_size': 32, 'f2_layer_size': 32, 'batch_size': 64, 'learning_rate': 0.0005, 'optimizer': 'adam', 'loss': 'Huber', 'dropout': 0.2, 'validation_split': 0.2, 'dataset': 'MNIST', 'architecture': 'CNN', 'dataset_size': 66976}
2000
{'epochs': 2000, 'classes': 8, 'f1_layer_size': 32, 'f2_layer_size': 32, 'batch_size': 64, 'learning_rate': 0.0005, 'optimizer': 'adam', 'loss': 'Huber', 'dropout': 0.2, 'validation_split': 0.2, 'dataset': 'MNIST', 'architecture': 'CNN', 'dataset_size': 66976}
Sequential(
  (0): Linear(in_features=120, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=8, bias=True)
  (5): Softmax(dim=1)
)


  0%|          | 1/2000 [00:22<12:21:49, 22.27s/it]

5    2052
4    1996
3    1965
2    1734
6    1626
1    1605
0    1367
7    1050
Name: class, dtype: int64 5    2052
4    1996
3    1965
2    1734
6    1626
1    1605
0    1367
7    1050
Name: class, dtype: int64


  1%|          | 21/2000 [01:38<4:57:40,  9.02s/it]

...

 55%|█████▌    | 1101/2000 [1:08:19<1:57:45,  7.86s/it]

5    2052
4    1996
3    1965
2    1734
6    1626
1    1605
0    1367
7    1050
Name: class, dtype: int64 5    2052
4    1996
3    1965
2    1734
6    1626
1    1605
0    1367
7    1050
Name: class, dtype: int64


 56%|█████▌    | 1121/2000 [1:09:26<1:55:07,  7.86s/it]

5    2052
4    1996
3    1965
2    1734
6    1626
1    1605
0    1367
7    1050
Name: class, dtype: int64 5    2052
4    1996
3    1965
2    1734
6    1626
1    1605
0    1367
7    1050
Name: class, dtype: int64


 56%|█████▌    | 1122/2000 [1:09:29<1:32:15,  6.31s/it]